In [1]:
############### Import packages
import os, numpy as np, pandas as pd, time, glob, re, math, statsmodels.api as sm, patsy as ps
from tqdm import tqdm
from time import process_time
from datetime import datetime
from datetime import date
from openpyxl import load_workbook
from patsy import dmatrices

############### Set working directory to parent directory
if os.getcwd() != 'F:\\github\\narrative_conservatism\\code':
    os.chdir('F:\\github\\narrative_conservatism\\code')

############### Set pandas column printing constraint
pd.set_option('display.max_columns', None)

In [2]:
#############################################################################################
####### Merge IBES annual data (oen-year forecast) with CRSP_COMP_EDGAR quarterly data ######
#############################################################################################

########### Read IBES raw data file
ibes_cols = ['CUSIP', 'VALUE', 'FPEDATS', 'ACTUAL']
ibes = pd.read_csv('..\\filings\\ibes.csv', usecols = ibes_cols)
print('number of cusip-fpedats-analyst: ' + str(ibes.shape[0]))

### Reorder and rename IBES columns
ibes = ibes.rename(columns={'CUSIP': 'cusip', 'VALUE': 'value', 'FPEDATS': 'fpedats', 'ACTUAL': 'actual'})
ibes = ibes[['cusip', 'fpedats', 'value', 'actual']]

### correct fpedats format
ibes['fpedats'] = ibes['fpedats'].str.replace('jan','01')
ibes['fpedats'] = ibes['fpedats'].str.replace('JAN','01')
ibes['fpedats'] = ibes['fpedats'].str.replace('feb','02')
ibes['fpedats'] = ibes['fpedats'].str.replace('FEB','02')
ibes['fpedats'] = ibes['fpedats'].str.replace('mar','03')
ibes['fpedats'] = ibes['fpedats'].str.replace('MAR','03')
ibes['fpedats'] = ibes['fpedats'].str.replace('apr','04')
ibes['fpedats'] = ibes['fpedats'].str.replace('APR','04')
ibes['fpedats'] = ibes['fpedats'].str.replace('may','05')
ibes['fpedats'] = ibes['fpedats'].str.replace('MAY','05')
ibes['fpedats'] = ibes['fpedats'].str.replace('jun','06')
ibes['fpedats'] = ibes['fpedats'].str.replace('JUN','06')
ibes['fpedats'] = ibes['fpedats'].str.replace('jul','07')
ibes['fpedats'] = ibes['fpedats'].str.replace('JUL','07')
ibes['fpedats'] = ibes['fpedats'].str.replace('aug','08')
ibes['fpedats'] = ibes['fpedats'].str.replace('AUG','08')
ibes['fpedats'] = ibes['fpedats'].str.replace('sep','09')
ibes['fpedats'] = ibes['fpedats'].str.replace('SEP','09')
ibes['fpedats'] = ibes['fpedats'].str.replace('oct','10')
ibes['fpedats'] = ibes['fpedats'].str.replace('OCT','10')
ibes['fpedats'] = ibes['fpedats'].str.replace('nov','11')
ibes['fpedats'] = ibes['fpedats'].str.replace('NOV','11')
ibes['fpedats'] = ibes['fpedats'].str.replace('dec','12')
ibes['fpedats'] = ibes['fpedats'].str.replace('DEC','12')
ibes['fpedats'] = pd.to_datetime(ibes['fpedats'], format='%d%m%Y')

print(ibes['fpedats'].describe())

### Delete missing cusip8 and actual
del_cusip = ibes[ibes['cusip'].isnull()].shape[0]
ibes = ibes[ibes['cusip'].isnull() == False]
print('number of obs. that contain missing cusip: ' + str(del_cusip))

del_actual = ibes[ibes['actual'].isnull()].shape[0]
ibes = ibes[ibes['actual'].isnull() == False]
print('number of obs. that contain missing actual: ' + str(del_actual))

### aggregate by cusip-fpedats and get actual, median, afe and consensus from ibes raw data
ibes_css = ibes.groupby(['cusip', 'fpedats'])['actual'].median().to_frame()
ibes_css['median'] = ibes.groupby(['cusip', 'fpedats'])['value'].median().to_frame()
ibes_css['afe'] = ibes_css['actual'] - ibes_css['median']
ibes_css['consensus'] = ibes.groupby(['cusip', 'fpedats'])['value'].mean().to_frame()
ibes_css['leap_consensus'] = ibes_css.groupby(['cusip'])['consensus'].shift(-1)

### make a consensus forecast dataset by cusip-fpedats
ibes = ibes.loc[ibes.duplicated(subset=['cusip', 'fpedats']) == False, ['cusip','fpedats']]
ibes = ibes.join(ibes_css, on=['cusip', 'fpedats'])

### create merge date_key - keep %y only
ibes['date_key'] = ibes['fpedats'].astype(str).str[:-6].astype(int)

print('number of cusip-fpedats, dropping missing cusip and actual: ' + str(ibes.shape[0]))

number of cusip-fpedats-analyst: 9927842
count                 9927842
unique                    418
top       2016-12-31 00:00:00
freq                   412685
first     1986-01-31 00:00:00
last      2023-07-31 00:00:00
Name: fpedats, dtype: object
number of obs. that contain missing cusip: 53373
number of obs. that contain missing actual: 1245510
number of cusip-fpedats, dropping missing cusip and actual: 151715


In [3]:
##################### Read EDGAR_CRSP_COMP and creat merge date_key
crsp_comp_edgar = pd.read_csv('..\\filings\\crsp_comp_edgar_10-Q.csv')
crsp_comp_edgar['date_key'] = crsp_comp_edgar['fyearq'].astype(int)

##################### Inner merge EDGAR_CRSP_COMP and IBES, key not unique in both datasets
crsp_comp_edgar_ibes = pd.merge(crsp_comp_edgar, ibes, on = ['cusip', 'date_key'], how='inner', validate = 'm:m')
print('number of cusip-date after merging: ' + str(crsp_comp_edgar_ibes.shape[0]))

crsp_comp_edgar_ibes.sort_values(by = ['cusip', 'cquarter'])

number of cusip-date after merging: 130777


,cusip,cik,rp,accnum,name,gvkey,SIC,fd,date_crsp,date_comp,cquarter,fyearq,fqtr,incorp,state,addzip,costat,age,actq,aqcy,cheq,dpq,ibq,intanq,lctq,revtq,txditcq,xsgaq,sstky,xrdq,atq,lag_atq,ceqq,lag_ceqq,cshoq,lag_cshoq,dlcq,lag_dlcq,dlttq,lag_dlttq,prccq,lag_prccq,ibq.1,lag_ibq,nw,READ,NW,TONE,TONE_GI,TONE_HE,TLAG,RET,STD_RET,EARN,LOSS,DEARN,STD_EARN,CFO,lag1_CFO,lag2_CFO,lag3_CFO,PPE,SIC2,SG,LAG_SG,SKEW_RET,TURNOVER,LAG1_RET,LAG2_RET,LAG3_RET,LAG1_NW,LAG2_NW,LAG3_NW,LAG1_TONE,LAG2_TONE,LAG3_TONE,LAG1_TLAG,LAG2_TLAG,LAG3_TLAG,NEG,SIZE,MTB,LEV,AGE,date_key,fpedats,actual,median,afe,consensus,leap_consensus
128527,00030710,1606180,2015-03-31,0001564590-15-003247,"AAC Holdings, Inc.",178698.0,8060,2015-05-05,2015-03-31,2015-03-31,201503,2015.0,1.0,NV,NV,37027,A,182,99.585,14.102,59.186,1.340,2.038,26.089,17.742,42.823,0.749,NaN,0.000,0.000,188.983,145.952,101.145,97.474,21.781,21.374,4.440,4.357,71.628,24.284,30.58,30.92,2.038,3.354,16287,24.89,9.698184,-7.183643,39.172346,6.324062,35.0,0.066935,0.234733,0.013963,0.0,-0.009017,NaN,-0.072387,0.021950,0.062143,NaN,0.484228,80,0.029934,0.003885,0.706554,93559.0,0.017252,NaN,NaN,9.899429,NaN,NaN,-8.534993,NaN,NaN,43.0,NaN,NaN,0,6.493578,6.780106,0.196236,5.209486,2015,2015-12-31,0.97,0.78,0.19,0.753333,1.015510
128528,00030710,1606180,2015-06-30,0001564590-15-005942,"AAC Holdings, Inc.",178698.0,8060,2015-08-03,2015-06-30,2015-06-30,201506,2015.0,2.0,NV,NV,37027,A,273,97.666,14.251,45.021,1.676,5.555,28.972,30.118,53.784,0.303,NaN,0.000,0.000,210.265,188.983,109.824,101.145,21.816,21.781,5.227,4.440,70.641,71.628,43.56,30.58,5.555,2.038,21532,27.31,9.977342,-10.821103,29.537433,7.895226,34.0,0.375746,0.011999,0.029394,0.0,0.018610,NaN,0.048258,-0.072387,0.021950,0.062143,0.491055,80,0.052129,0.029934,0.067869,126640.0,0.066935,0.017252,NaN,9.698184,9.899429,NaN,-7.183643,-8.534993,NaN,35.0,43.0,NaN,0,6.501384,6.585229,0.402512,5.613128,2015,2015-12-31,0.97,0.78,0.19,0.753333,1.015510
128529,00030710,1606180,2015-09-30,0001564590-15-010373,"AAC Holdings, Inc.",178698.0,8060,2015-11-10,2015-09-30,2015-09-30,201509,2015.0,3.0,NV,NV,37027,A,365,76.724,84.471,10.764,1.921,2.452,112.594,32.120,57.372,1.942,NaN,0.000,0.000,290.911,210.265,139.720,109.824,22.407,21.816,4.856,5.227,116.710,70.641,22.25,43.56,2.452,5.555,21050,29.19,9.954703,-8.123515,32.684086,7.933492,41.0,-0.482160,0.121349,0.011661,0.0,-0.014758,0.009461,-0.003657,0.048258,-0.072387,0.021950,0.518812,80,0.012334,0.052129,-0.256675,399243.0,0.375746,0.066935,0.017252,9.977342,9.698184,9.899429,-10.821103,-7.183643,-8.534993,34.0,35.0,43.0,1,6.856783,8.652981,0.360821,5.902633,2015,2015-12-31,0.97,0.78,0.19,0.753333,1.015510
128530,00030710,1606180,2016-03-31,0001564590-16-018317,"AAC Holdings, Inc.",178698.0,8060,2016-05-06,2016-03-31,2016-03-31,201603,2016.0,1.0,NV,NV,37027,A,548,81.832,0.550,13.237,3.915,0.586,117.858,31.397,65.348,1.195,NaN,0.000,0.000,314.995,316.049,145.136,141.654,22.969,22.814,4.092,4.806,138.996,140.335,19.79,19.06,0.586,0.448,11675,27.81,9.365291,-8.822270,21.498929,4.111349,36.0,0.035246,0.099557,0.001854,0.0,0.000437,0.011384,-0.006056,0.001268,-0.003657,0.048258,0.411927,80,0.022432,0.002879,0.259842,165281.0,-0.177036,-0.482160,0.375746,9.954703,9.977342,9.698184,-8.123515,-10.821103,-7.183643,41.0,34.0,35.0,0,6.074966,3.069697,0.459236,6.308098,2016,2016-12-31,0.71,0.98,-0.27,1.015510,0.813794
128531,00030710,1606180,2016-06-30,0001564590-16-022835,"AAC Holdings, Inc.",178698.0,8060,2016-08-05,2016-06-30,2016-06-30,201606,2016.0,2.0,NV,NV,37027,A,639,85.688,19.150,7.269,4.225,0.872,144.637,37.299,71.542,1.050,NaN,0.000,0.000,362.200,314.995,163.576,145.136,23.696,22.969,4.702,4.092,162.821,138.996,22.82,19.79,0.872,0.586,14997,27.30,9.615672,-7.468160,21.004201,7.534840,36.0,0.116765,0.013363,0.002768,0.0,0.000908,0.011912,0.006473,-0.006056,0.001268,-0.003657,0.478462,80,0.017101,0.022432,0.697928,101078.0,0.035246,-0.177036,-0.482160,9.365291,9.954703,9.977342,-8.822270,-8.123515,-10.821103,

In [4]:
########################################################################################
############ Merge CRSP_COMP_EDGAR_IBES data with SEG data #############################
########################################################################################

########### Read SEG raw data file
seg_cols = ['gvkey', 'stype', 'sid', 'datadate', 'snms', 'cusip', 'cik']
seg = pd.read_csv('..\\filings\\compustat_seg.csv', usecols = seg_cols)
print('number of cusip-fpedats-analyst: ' + str(seg.shape[0]))

### Reorder and rename IBES columns
seg = seg[['gvkey', 'cik', 'cusip', 'datadate', 'stype', 'sid', 'snms']]
seg = seg.rename(columns={'datadate': 'date_comp'})

### parse date_comp format
seg['date_comp'] = pd.to_datetime(seg['date_comp'], format='%Y%m%d')

### aggregate by gvkey-date_comp-stype and get count of sid
seg_count = seg.groupby(['gvkey', 'date_comp', 'stype'])['sid'].count().to_frame()

### make a segment dataset by gvkey-date_comp-stype
seg = seg.loc[seg.duplicated(subset=['gvkey', 'date_comp', 'stype']) == False, ['gvkey', 'date_comp', 'stype']]
seg = seg.join(seg_count, on=['gvkey', 'date_comp', 'stype'])
seg_bus = seg[seg['stype']=='BUSSEG']
seg_bus = seg_bus.drop(columns=['stype'])
seg_bus = seg_bus.rename(columns={'sid': 'nseg_bus'})
seg_geo = seg[seg['stype']=='GEOSEG']
seg_geo = seg_geo.drop(columns=['stype'])
seg_geo = seg_geo.rename(columns={'sid': 'nseg_geo'})
seg = seg.loc[seg.duplicated(subset=['gvkey', 'date_comp']) == False, ['gvkey', 'date_comp']]
seg = pd.merge(seg, seg_bus, on = ['gvkey', 'date_comp'], how='left', validate = '1:1')
seg = pd.merge(seg, seg_geo, on = ['gvkey', 'date_comp'], how='left', validate = '1:1')
seg = seg.sort_values(by=['gvkey', 'date_comp'])
seg.loc[seg['nseg_geo'].isnull(), 'nseg_geo'] = 1
seg.loc[seg['nseg_bus'].isnull(), 'nseg_bus'] = 1
seg['nseg_bus'] = seg['nseg_bus'].astype(int)
seg['nseg_geo'] = seg['nseg_geo'].astype(int)

### create merge date_key - %y only
seg['date_key'] = seg['date_comp'].astype(str).str[:-6].astype(int)
seg = seg.drop(columns=['date_comp'])

print('number of gvkey-datadate in SEGMENT data: ' + str(seg.shape[0]))

number of cusip-fpedats-analyst: 452653
number of gvkey-datadate in SEGMENT data: 50876


In [5]:
##################### left merge EDGAR_CRSP_COMP_IBES and SEG, key not unique in both datasets
crsp_comp_edgar_ibes_seg = pd.merge(crsp_comp_edgar_ibes, seg, on = ['gvkey', 'date_key'], how='left', validate = 'm:m')
# crsp_comp_edgar_ibes_seg_inner = pd.merge(crsp_comp_edgar_ibes, seg, on = ['gvkey', 'date_key'], how='inner', validate = 'm:m')
# print('Number of gvkey-quarter obs. after merging, inner: '+ str(crsp_comp_edgar_ibes_seg_inner.shape[0]))

##################### Fill missing segments in crsp_comp_edgar_ibes_seg_left with 1
crsp_comp_edgar_ibes_seg.loc[crsp_comp_edgar_ibes_seg['nseg_bus'].isnull(), 'nseg_bus'] = 1
crsp_comp_edgar_ibes_seg.loc[crsp_comp_edgar_ibes_seg['nseg_geo'].isnull(), 'nseg_geo'] = 1

####### Drop duplicated gykey-cquarter
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg.duplicated(subset=['gvkey', 'cquarter']) == False]
print('Number of gvkey-quarter obs. after merging, left: '+ str(crsp_comp_edgar_ibes_seg.shape[0]))

Number of gvkey-quarter obs. after merging, left: 130745


In [6]:
########################################################################################
############################### Variable Creation ######################################
########################################################################################

################### CRSP_COMP_EDGAR_IBES_SEG_LEFT ######################################

######## AGE: log(1 + age from the first year the firm entered the CRSP dataset)
crsp_comp_edgar_ibes_seg['AGE'] = np.log(1 + crsp_comp_edgar_ibes_seg['age'])

######## BUSSEG: log(1 + number of business segments), or 1 if item is missing from Compustat; and
######## GEOSEG: log(1 + number of geographic segments), or 1 if item is missing from Compustat.
crsp_comp_edgar_ibes_seg['BUSSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg['nseg_bus'])
crsp_comp_edgar_ibes_seg['GEOSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg['nseg_geo'])

######## AFE, analyst forecast error, \
######## defined as I/B/E/S earnings per share minus the median of the most recent analysts’ forecasts, \
######## deflated by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg['AFE'] = crsp_comp_edgar_ibes_seg['afe']/crsp_comp_edgar_ibes_seg['prccq']

######## AF, analyst consensus forecast for one-year-ahead earnings per share, scaled by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg['AF'] = crsp_comp_edgar_ibes_seg['leap_consensus']/crsp_comp_edgar_ibes_seg['prccq']

In [7]:
########################################################################################
########################## Variable Screening ##########################################
########################################################################################

########## Drop files (firm-quarter) that have positive/negative infinity DEARN
del_DEARN = crsp_comp_edgar_ibes_seg[(crsp_comp_edgar_ibes_seg['DEARN']==np.inf) | (crsp_comp_edgar_ibes_seg['DEARN']==-np.inf)].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[(crsp_comp_edgar_ibes_seg['DEARN']!=np.inf) & (crsp_comp_edgar_ibes_seg['DEARN']!=-np.inf)]
print('number of files that have positive/negative infinity DEARN: ' + str(del_DEARN))

### Drop missing AF
del_AF = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AF'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AF'].isnull() == False]
print('number of obs. that contain missing AF: ' + str(del_AF))

### Drop missing AFE
del_AFE = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AFE'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AFE'].isnull() == False]
print('number of obs. that contain missing AFE: ' + str(del_AFE))

### Drop missing EARN
del_EARN = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['EARN'].isnull() == False]
print('number of obs. that contain missing EARN: ' + str(del_EARN))

### Drop missing STD_EARN
del_STD_EARN = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['STD_EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['STD_EARN'].isnull() == False]
print('number of obs. that contain missing STD_EARN: ' + str(del_STD_EARN))

### Drop missing EARN
del_DEARN = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DEARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DEARN'].isnull() == False]
print('number of obs. that contain missing DEARN: ' + str(del_DEARN))

############## Winsorize ##############################
###### Define a function that winsorize a variable at 1% and 99% 
def winsorize (df, colnames):
    for col in colnames:
        varq01 = df[col].quantile(.01)
        varq99 = df[col].quantile(.99)
        df[col] = df[col].clip(varq01, varq99)
    return df

############## Winsorize TABLE 1 (line 1) and TABLE 4 (line 2) variables
crsp_comp_edgar_ibes_seg = winsorize(crsp_comp_edgar_ibes_seg, \
                                          ['AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'DEARN', 'STD_EARN', 'STD_RET', 'SIZE', 'MTB', 'LEV', \
                                          'CFO', 'lag1_CFO', 'lag2_CFO', 'lag3_CFO', 'READ'])

############## Inspect sample size after variable screening
print('Number of firm-quarters after variable screening: ' + str(crsp_comp_edgar_ibes_seg.shape[0]))

number of files that have positive/negative infinity DEARN: 0
number of obs. that contain missing AF: 16704
number of obs. that contain missing AFE: 0
number of obs. that contain missing EARN: 69
number of obs. that contain missing STD_EARN: 3687
number of obs. that contain missing DEARN: 0
Number of firm-quarters after variable screening: 110285


In [8]:
# ######## Create ABTONE: residual from TONE regression 
# y, X = ps.dmatrices('TONE ~ 1 + EARN + RET + SIZE + MTB + STD_RET + STD_EARN + AGE + BUSSEG + GEOSEG + LOSS + DEARN + AFE + AF', \
#                     data = crsp_comp_edgar_ibes_seg, return_type = 'dataframe')
# model = sm.OLS(y, X)
# res = model.fit()
# # res.summary()
# crsp_comp_edgar_ibes_seg['ABTONE'] = res.resid

# ######## Create abtone for REPLICATION: residual from tone regression 
# y, X = ps.dmatrices('tone ~ 1 + EARN + RET + SIZE + MTB + STD_RET + STD_EARN + AGE + BUSSEG + GEOSEG + LOSS + DEARN + AFE + AF', \
#                     data = crsp_comp_edgar_ibes_seg, return_type = 'dataframe')
# model = sm.OLS(y, X)
# res = model.fit()
# # res.summary()
# crsp_comp_edgar_ibes_seg['abtone'] = res.resid

############## Save merged CRSP_COMP_EDGAR_IBES_SEG to csv file
crsp_comp_edgar_ibes_seg.to_csv('..\\filings\\crsp_comp_edgar_ibes_seg_10-Q.csv', index = 0)

crsp_comp_edgar_ibes_seg

,cusip,cik,rp,accnum,name,gvkey,SIC,fd,date_crsp,date_comp,cquarter,fyearq,fqtr,incorp,state,addzip,costat,age,actq,aqcy,cheq,dpq,ibq,intanq,lctq,revtq,txditcq,xsgaq,sstky,xrdq,atq,lag_atq,ceqq,lag_ceqq,cshoq,lag_cshoq,dlcq,lag_dlcq,dlttq,lag_dlttq,prccq,lag_prccq,ibq.1,lag_ibq,nw,READ,NW,TONE,TONE_GI,TONE_HE,TLAG,RET,STD_RET,EARN,LOSS,DEARN,STD_EARN,CFO,lag1_CFO,lag2_CFO,lag3_CFO,PPE,SIC2,SG,LAG_SG,SKEW_RET,TURNOVER,LAG1_RET,LAG2_RET,LAG3_RET,LAG1_NW,LAG2_NW,LAG3_NW,LAG1_TONE,LAG2_TONE,LAG3_TONE,LAG1_TLAG,LAG2_TLAG,LAG3_TLAG,NEG,SIZE,MTB,LEV,AGE,date_key,fpedats,actual,median,afe,consensus,leap_consensus,nseg_bus,nseg_geo,BUSSEG,GEOSEG,AFE,AF
0,48273010,20,1995-09-30,0000893220-95-000710,K TRON INTERNATIONAL INC,6314.0,3823,1995-11-03,1995-09-29,1995-09-30,199509,1995.0,3.0,NJ,NJ,08071-0888,I,5418,48.464,0.000,2.507,0.974,0.375,NaN,63.869,22.297,0.199,7.615,0.074,0.535,75.132,76.265,8.439,8.152,3.104,3.104,41.114,42.890,0.189,0.225,5.500,5.50,0.375,-8.950,2770,60.33,7.926964,-10.108303,21.299639,7.581227,34.0,-0.171164,0.038383,0.004917,0.0,0.122271,0.034862,0.033764,0.006208,-0.008558,0.014989,NaN,38,-0.142216,0.032505,0.059820,910.0,-0.079184,-0.583745,-0.030765,7.820440,9.328123,NaN,-12.449799,-14.845764,NaN,45.0,45.0,NaN,1,2.955028,2.094210,0.565331,8.597667,1995,1995-12-31,-3.01,0.100,-3.110,-0.243750,0.824286,1.0,1.0,0.693147,0.693147,-0.443892,0.149870
1,48273010,20,1996-03-30,0000893220-96-000686,K TRON INTERNATIONAL INC,6314.0,3823,1996-05-02,1996-03-29,1996-03-31,199603,1996.0,1.0,NJ,NJ,08071-0888,I,5600,42.388,0.000,2.450,0.796,0.752,NaN,26.358,23.579,0.466,8.125,0.000,0.613,65.438,69.296,9.897,9.421,3.113,3.113,7.882,2.133,26.593,35.004,7.750,6.25,0.752,0.561,2614,103.24,7.869019,-8.033665,22.953328,8.798776,33.0,0.197571,0.156629,0.010852,0.0,0.002756,0.034785,-0.068547,0.048288,0.033764,0.006208,0.633096,38,-0.015786,0.033407,0.524379,2519.0,0.118911,-0.171164,-0.079184,7.926964,7.820440,9.328123,-10.108303,-12.449799,-14.845764,34.0,45.0,45.0,0,2.968168,2.065200,0.535918,8.630700,1996,1996-12-31,1.29,0.820,0.470,0.824286,1.334444,1.0,1.0,0.693147,0.693147,0.060645,0.172186
2,48273010,20,1996-06-29,0000893220-96-001241,K TRON INTERNATIONAL INC,6314.0,3823,1996-07-26,1996-06-28,1996-06-30,199606,1996.0,2.0,NJ,NJ,08071-0888,I,5691,39.157,0.000,3.325,0.771,0.928,NaN,45.448,21.880,0.466,8.025,0.070,0.619,61.142,65.438,10.592,9.897,3.127,3.113,27.322,7.882,2.725,26.593,8.500,7.75,0.928,0.752,3145,71.90,8.053887,-7.949126,21.939587,4.133545,27.0,0.053657,0.054066,0.014181,0.0,0.002690,0.036744,0.070127,-0.068547,0.048288,0.033764,0.656346,38,-0.027788,-0.015786,-0.518552,3788.0,0.197571,0.118911,-0.171164,7.869019,7.926964,7.820440,-8.033665,-10.108303,-12.449799,33.0,34.0,45.0,0,3.183280,2.437683,0.526835,8.646817,1996,1996-12-31,1.29,0.820,0.470,0.824286,1.334444,1.0,1.0,0.693147,0.693147,0.055294,0.156993
3,48273010,20,1996-09-28,0000893220-96-001772,K TRON INTERNATIONAL INC,6314.0,3823,1996-10-30,1996-09-30,1996-09-30,199609,1996.0,3.0,NJ,NJ,08071-0888,I,5785,37.346,0.000,2.253,0.894,1.161,NaN,42.412,22.547,0.466,7.668,0.070,0.523,59.275,61.142,11.824,10.592,3.127,3.127,23.326,27.322,2.772,2.725,9.250,8.50,1.161,0.928,12335,215.18,9.420277,-9.809485,22.294285,5.755979,32.0,0.055580,0.021281,0.018989,0.0,0.003811,0.005546,0.060155,0.070127,-0.068547,0.048288,0.722482,38,0.011253,-0.027788,-0.546126,5791.0,0.053657,0.197571,0.118911,8.053887,7.869019,7.926964,-7.949126,-8.033665,-10.108303,27.0,33.0,34.0,0,3.280140,2.509394,0.491430,8.663196,1996,1996-12-31,1.29,0.820,0.470,0.824286,1.334444,1.0,1.0,0.693147,0.693147,0.050811,0.144264
4,48273010,20,1997-03-29,0000893220-97-000850,K TRON INTERNATIONAL INC,6314.0,3823,1997-04-30,1997-03-31,1997-03-31,199703,1997.0,1.0,NJ,NJ,08071-0888,I,5967,33.912,0.000,4.426,0.770,1.050,NaN,19.139,21.344,0.459,7.684,0.050,0.722,53.037,55.330,13.770,13.194,3.143,3.137,0.325,0.861,18.316,20.807,10.375,10.25,1.050,1.185,1756,26.60,7.471363,-2.277904,21.640091,-3.416856,32.0,0.006413,0.

In [9]:
# ####################################################################################################################
# ############# Create COMP_DA: if not replicating TABLE 4, DO NOT run because cause large sample reduction ##########
# ####################################################################################################################

# ### Drop missing or infinite LAG_TA_REV, DSAR and PPE
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg.loc[(crsp_comp_edgar_ibes_seg['LAG_TA_REV'] != np.inf) & \
#                                                         (crsp_comp_edgar_ibes_seg['LAG_TA_REV'].isnull() == False) & \
#                                                         (crsp_comp_edgar_ibes_seg['LAG_TA_REV'] != -np.inf)]
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg.loc[(crsp_comp_edgar_ibes_seg['DSAR'] != np.inf) & \
#                                                         (crsp_comp_edgar_ibes_seg['DSAR'].isnull() == False) & \
#                                                         (crsp_comp_edgar_ibes_seg['DSAR'] != -np.inf)]
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg.loc[(crsp_comp_edgar_ibes_seg['PPE'] != np.inf) & \
#                                                         (crsp_comp_edgar_ibes_seg['PPE'].isnull() == False) & \
#                                                         (crsp_comp_edgar_ibes_seg['PPE'] != -np.inf)]
# print('number of obs. after deleting missing or infinite LAG_TA_REV, DSAR and PPE: ' + str(crsp_comp_edgar_ibes_seg.shape[0]))

# ############## Winsorize DA variables
# crsp_comp_edgar_ibes_seg = winsorize(crsp_comp_edgar_ibes_seg, ['TACC', 'LAG_TA_REV', 'DSAR', 'PPE'])

# ######## Create DA: residual from TACC regression by each two-digit SIC-year
# def TACC_reg(data):
#     try:
#         y, X = ps.dmatrices('TACC ~ 1 + LAG_TA_REV + DSAR + PPE', data = data, return_type = 'dataframe')
#         model = sm.OLS(y, X)
#         res = model.fit()
#         data['DA'] = res.resid
#         return data
#     except:
#         pass

# crsp_comp_edgar_ibes_seg1 = crsp_comp_edgar_ibes_seg.groupby(['SIC2','fyearq']).apply(TACC_reg)
# # comp_DA['DA'].describe()

# ####### Join DA to COMP
# crsp_comp_edgar_ibes_seg = pd.merge(crsp_comp_edgar_ibes_seg, crsp_comp_edgar_ibes_seg1[['cik', 'rp', 'DA']], on = ['cik', 'rp'], how='left', validate = '1:1')

# crsp_comp_edgar_ibes_seg

# ############## Dropping missing DA to replicate Huang et al. 2014 TABLE 4, \
# del_DA = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DA'].isnull()].shape[0]
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DA'].isnull() == False]
# print('number of obs. that contain missing DA: ' + str(del_DA))

# ############## Inspect sample size after variable screening
# print('Number of firm-quarters after dropping missing DA: ' + str(crsp_comp_edgar_ibes_seg.shape[0]))

# ############## Save merged ID_CRSP_COMP_TEXT to csv file
# crsp_comp_edgar_ibes_seg.to_csv('..\\filings\\crsp_comp_edgar_ibes_seg_DA_10-Q.csv', index = 0)

In [9]:
########################################################################################
########### Table 2 - Panel A: Summary Statistics and Correlation Matrix ###############
########################################################################################

############# Table 2 Panel A: Summary statistics for selected variables
######### Variable groups:
# 1st line: textual variables, generally consistent with LM's summary statistics
# 2nd line: fundamental variables (main)
# 3rd line: abtone
selected_vars = crsp_comp_edgar_ibes_seg[['NW','nw', 'TONE','TONE_GI', 'TONE_HE', 'TLAG', 'READ', \
                                          'RET', 'NEG', 'SIZE', 'MTB', 'LEV', \
                                          'AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'STD_EARN', 'STD_RET', 'LOSS'\
                                        # 'DEARN', 'LOSS', ABTONE', 'DA' 
                                        ]]

T2PA = selected_vars.describe().transpose() 

############ count number of unique firms
print('Number of unique firms in final 10-Q sample: ' + str(crsp_comp_edgar_ibes_seg['cik'].unique().shape[0]))

############# Summary statistics for all raw and processed variables
full_summary = crsp_comp_edgar_ibes_seg.describe().transpose()

############# Save T3PA
table_path = '..\\output\\Tables.xlsx'
if os.path.exists(table_path) == True:
    book = load_workbook(table_path)
    writer = pd.ExcelWriter(table_path, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    T2PA.to_excel(writer, sheet_name='T2PA_raw', float_format="%.4f")

    writer.save()
    writer.close()
    
else:
    T2PA.to_excel(table_path, sheet_name='T2PA_raw', float_format="%.4f")

T2PA

Number of unique firms in final 10-Q sample: 5957


,count,mean,std,min,25%,50%,75%,max
NW,110285.0,9.122917,0.750859,7.120444,8.649799,9.203014,9.632990,13.544145
nw,110285.0,11963.999456,10242.279660,1236.000000,5708.000000,9926.000000,15259.000000,762337.000000
TONE,110285.0,-8.911503,7.182816,-63.578565,-13.064339,-7.760962,-3.873358,24.214907
TONE_GI,110285.0,24.436153,6.970644,-21.510297,19.872943,24.278565,28.771655,93.200190
TONE_HE,110285.0,6.429321,5.024306,-26.068577,3.359227,5.959983,8.998026,55.884287
TLAG,110285.0,38.300322,6.069524,0.000000,35.000000,39.000000,44.000000,51.000000
READ,110285.0,36.611752,40.573468,14.590000,17.850000,20.050000,35.210000,255.238000
RET,110285.0,0.016987,0.247440,-1.594595,-0.110579,0.006548,0.125804,4.849226
NEG,110285.0,0.484526,0.499763,0.000000,0.000000,0.000000,1.000000,1.000000
SIZE,110285.0,6.605742,1.814639,2.955028,5.298486,6.472564,7.762951,11.506680


In [10]:
# full_summary

In [11]:
############# Table 3 Panel B: Correlation matrix for selected variables
######### pearson correlation
T2PC_pearson = selected_vars.loc[:, (selected_vars.columns != 'nw') & \
                                 (selected_vars.columns != 'BUSSEG') & \
                                 (selected_vars.columns != 'GEOSEG') & \
                                 (selected_vars.columns != 'LOSS')].corr(method='pearson')

# T2PC_pearson

In [12]:
######### spearman correlation
T2PC_spearman = selected_vars.loc[:, (selected_vars.columns != 'nw') & \
                                 (selected_vars.columns != 'BUSSEG') & \
                                 (selected_vars.columns != 'GEOSEG') & \
                                 (selected_vars.columns != 'LOSS')].corr(method='spearman')

# T2PC_spearman

In [13]:
######### Combine two correlation matrices. right-up matrix: pearson; left-down matrix: spearman 
for row in list(range(0, len(T2PC_spearman.index))):
    T2PC_spearman.iloc[row, row+1:] = T2PC_pearson.iloc[row, row+1:]
    
##### Save T2PC
table_path = '..\\output\\Tables.xlsx'
if os.path.exists(table_path) == True:
    book = load_workbook(table_path)
    writer = pd.ExcelWriter(table_path, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    T2PC_spearman.to_excel(writer, sheet_name='T2PC_raw', float_format="%.4f")

    writer.save()
    writer.close()
    
else:
    T2PC_spearman.to_excel(table_path, sheet_name='T2PC_raw', float_format="%.4f")

T2PC_spearman

,NW,TONE,TONE_GI,TONE_HE,TLAG,READ,RET,NEG,SIZE,MTB,LEV,AF,AFE,AGE,EARN,STD_EARN,STD_RET
NW,1.000000,-0.441355,0.011296,-0.243330,-0.235017,-0.099376,-0.006539,0.001315,0.271327,0.087204,0.064618,-0.107308,0.021371,-0.049915,-0.153618,0.101737,-0.032690
TONE,-0.454376,1.000000,0.419295,0.363428,0.009136,0.083251,0.015417,-0.014719,-0.052310,-0.021461,0.076021,0.090104,0.090078,0.076783,0.163402,-0.143439,-0.094050
TONE_GI,-0.008045,0.395980,1.000000,0.158782,0.007376,0.081290,0.017625,-0.013139,-0.013475,0.075283,0.145233,-0.039093,0.070887,-0.065359,-0.012864,0.001486,-0.036908
TONE_HE,-0.213705,0.379338,0.169254,1.000000,0.013900,0.051229,0.035037,-0.052517,0.128538,0.097995,0.033918,0.084773,0.174968,0.019200,0.225289,-0.154236,-0.097676
TLAG,-0.298748,0.015170,-0.004138,0.018025,1.000000,0.060357,-0.018529,0.035214,-0.375929,-0.055912,-0.027574,-0.078818,-0.126947,-0.234837,-0.114711,0.117317,0.199246
READ,-0.234789,0.158481,0.128815,0.058795,0.134200,1.000000,-0.013637,0.012444,-0.014593,-0.042293,0.060264,0.055235,0.001750,0.089462,0.067523,-0.048228,-0.044398
RET,-0.007339,0.022766,0.016794,0.059137,-0.033512,-0.024148,1.000000,-0.683886,-0.062647,-0.018344,-0.004933,-0.006173,0.148491,-0.003474,0.048762,0.012311,0.261587
NEG,0.003017,-0.017488,-0.013876,-0.056163,0.035481,0.024099,-0.865611,1.000000,-0.002179,0.004720,0.002333,0.005106,-0.118886,-0.013480,-0.063298,0.016599,-0.112820
SIZE,0.279029,-0.035708,-0.005504,0.142556,-0.382008,-0.077295,-0.022064,-0.002882,1.000000,0.242736,0.142429,0.130132,0.252649,0.345577,0.262645,-0.206237,-0.325987
MTB,0.072869,0.033223,0.090494,0.199357,-0.077407,-0.038535,-0.043045,0.022650,0.374965,1.000000,0.072589,-0.134043,0.119101,-0.089708,-0.062379,0.149833,0.016463
